In [54]:
import mysql.connector

import pandas as pd
import time
import dateutil




def doQuerySql(db_nm, sql, sqlData=""):
    rows = []
    if db_nm == "analyze_db":
        cnx = mysql.connector.connect(user='jinyao', database='ajmide_analyze',
                                      host='rdsua5i7pkip9jx53dwgb.mysql.rds.aliyuncs.com', password='jy1008', port=3306)

    if db_nm == "analyze_stat":
        cnx = mysql.connector.connect(user='jinyao', database='ajmide_stat',
                                      host='rdsua5i7pkip9jx53dwgb.mysql.rds.aliyuncs.com', password='jy1008', port=3306)

    if db_nm == "main_db":
        #        cnx = mysql.connector.connect(user='jinyao', database='ajmide_main', host='rdsfuff2efuff2e.mysql.rds.aliyuncs.com', password='jy1008',port=3306)
        cnx = mysql.connector.connect(user='ajmide_stat_read', database='ajmide_main_m',
                                      host='rdsy67za3yvibvq.mysql.rds.aliyuncs.com', password='vnhf762Hfg3', port=3306)

    if db_nm == "main_db_read":
        cnx = mysql.connector.connect(user='ajmide_stat_read', database='ajmide_main_m',
                                      host='rds30s9go4s854r689d8.mysql.rds.aliyuncs.com', password='vnhf762Hfg3',
                                      port=3306)
        # 执行sql并返回结果集
    if db_nm == "plug":
        cnx = mysql.connector.connect(user='jinyao', database='ajmide_plug',
                                      host='rdsb8wer169x4v95m28k.mysql.rds.aliyuncs.com', password='Opera@0329',
                                      port=3306)

    if db_nm == "ods_dw":
        cnx = mysql.connector.connect(user='jinyao', database='ajmide_dw',
                                      host='120.27.141.126', password='jinyao',
                                      port=3306)

    print(cnx)
    while True:
        rows = []
        try:
            cursor = cnx.cursor()
            cursor.execute(sql, sqlData)
            for row in cursor.fetchall():
                # with column name
                #rows.append(dict(zip(cursor.column_names,row)))
                rows.append(row)
            cursor.close()
            break
        except mysql.connector.Error as err:
            print(err)
            pass
        time.sleep(1)
    return rows


def func_bin(row,col, config_dict):
                
        if row[col] < config_dict['b1']:
            ind = '09_LL'
        elif row[col] < config_dict['b2']:
            ind = '08_LM'
        elif row[col] < config_dict['b3']:
            ind = '07_LH'
        elif row[col] < config_dict['b4']:
            ind = '06_ML'
        elif row[col] < config_dict['b5']:
            ind = '05_MM'
        elif row[col] < config_dict['b6']:
            ind = '04_MH'
        elif row[col] < config_dict['b7']:
            ind = '03_HL'
        elif row[col] < config_dict['b8']:
            ind = '02_HM'
        else:
            ind = '01_HH'
        return ind
    

def define_cnx(mysql_mode, database):

    cnx=''

    if mysql_mode == "analyze":
        cnx = mysql.connector.connect(user='jinyao', database=database,
                                      host='rdsua5i7pkip9jx53dwgb.mysql.rds.aliyuncs.com', password='jy1008', port=3306)

    if mysql_mode == "ods":
        cnx = mysql.connector.connect(user='jinyao', database=database,
                                  host='120.27.141.126', password='jinyao',
                                  port=3306)
    if mysql_mode == "main":
        cnx = mysql.connector.connect(user='ajmide_stat_read', database=database,
                                      host='rds30s9go4s854r689d8.mysql.rds.aliyuncs.com', password='vnhf762Hfg3',
                                      port=3306)
    return cnx

if __name__ == '__main__':

    
    
    sql="""
        SELECT
          pid,
          daily_new_fans,
          daily_pid_device_count,
          daily_pid_radio_score,
          pid_absolute_score,
          pid_relative_score,
          daily_short_radio_score,
          presenter_score     
        FROM
          ajmide_dw.stat_pid_daily
        where
           pid_date >= '2017-05-16' and  pid_date < '2017-05-17'"""

    cnx = define_cnx("ods", "ajmide_dw")

    list_to_qtl = ['daily_new_fans', 
                   'daily_pid_device_count',
                  'daily_pid_radio_score',
                  'pid_absolute_score',
                  'pid_relative_score',
                  'daily_short_radio_score',
                  'presenter_score']

    df = pd.read_sql(sql,cnx)
   

In [55]:
    print("社区日活: daily_pid_device_count is started")
    oo = df.ix[(df['daily_pid_device_count'] >50)][list_to_qtl]

    df_tmp = oo.quantile([0, .1, .2, .3, .4, .5, .6, .7, .8, .9, 0.95, 0.99, 1  ])
    df_tmp
    
    config_dict= {            
            "b1":50,
            "b2":100,
            "b3":300,
            "b4":500,
            "b5":800,
            "b6":1200,
            "b7":2000,
            "b8":3000
        }
    
    df['bin_device_count'] = df.apply (lambda row: func_bin(row, 'daily_pid_device_count', config_dict),axis=1)
    
    
    aggregations={'daily_pid_device_count':
                                             {
                                             'avg_device_count':'mean',
                                             'sum_device_count':'sum',
                                             'cnt_pid':'count'
                                             }
                    }
    df.groupby(['bin_device_count']).agg(aggregations)


社区日活: daily_pid_device_count is started


daily_pid_device_count                         
                       avg_device_count sum_device_count cnt_pid
bin_device_count                                                
01_HH                       4371.000000            26226       6
02_HM                       2419.250000             9677       4
03_HL                       1444.857143            10114       7
04_MH                        951.400000             9514      10
05_MM                        623.636364            13720      22
06_ML                        376.500000            12801      34
07_LH                        172.333333            19129     111
08_LM                         68.990826             7520     109
09_LL                          4.304015            18546    4309

In [56]:
    
    
    spc_var = 'pid_absolute_score'
    print("社区绝对活跃度: {s} is started".format(s=spc_var))
    oo = df.ix[(df[spc_var] >0)][list_to_qtl]
    
    oo.quantile([0, .1, .2, .3, .4, .5, .6, .7, .8, .9, 0.95, 0.99, 1  ])
    
    
    config_dict= {            
            "b1":1,
            "b2":10,
            "b3":20,
            "b4":30,
            "b5":50,
            "b6":100,
            "b7":150,
            "b8":200
        }
    
    df["""bin_{spc_var}""".format(spc_var=spc_var)] = df.apply (lambda row: func_bin(row, spc_var, config_dict),axis=1)
    
    
    aggregations={spc_var:
                                             {
                                             'avg':'mean',
                                             'sum':'sum',
                                             'cnt_pid':'count'
                                             }
                    }
    df.groupby(["""bin_{spc_var}""".format(spc_var=spc_var)]).agg(aggregations)

社区绝对活跃度: pid_absolute_score is started


pid_absolute_score                
                                      avg     sum cnt_pid
bin_pid_absolute_score                                   
01_HH                          536.150000  3216.9       6
02_HM                          167.000000   334.0       2
03_HL                          118.180000   590.9       5
04_MH                           74.763636   822.4      11
05_MM                           41.672222   750.1      18
06_ML                           22.677273   498.9      22
07_LH                           13.600000   503.2      37
08_LM                            3.208421  1219.2     380
09_LL                            0.027717   114.5    4131

In [60]:
    spc_var = 'pid_relative_score'
    print("社区相对活跃度: {s} is started".format(s=spc_var))
    oo = df.ix[(df['daily_pid_device_count']>0) & (df['pid_relative_score']<1) ][list_to_qtl]
    
    oo.quantile([0, .1, .2, .3, .4, .5, .6, .7, .8, .9, 0.95, 0.99, 1  ])
    
    config_dict= {            
            "b1":0.001,
            "b2":0.005,
            "b3":0.1,
            "b4":0.2,
            "b5":0.3,
            "b6":0.4,
            "b7":0.5,
            "b8":0.6
        }
    
    df["""bin_{spc_var}""".format(spc_var=spc_var)] = df.apply (lambda row: func_bin(row, spc_var, config_dict),axis=1)
    
    
    aggregations={spc_var:
                                             {
                                             'avg':'mean',
                                             'sum':'sum',
                                             'cnt_pid':'count'
                                             }
                    }
    print(df.groupby(["""bin_{spc_var}""".format(spc_var=spc_var)]).agg(aggregations))
    
    df.count()


社区相对活跃度: pid_relative_score is started
                       pid_relative_score                    
                                      avg         sum cnt_pid
bin_pid_relative_score                                       
01_HH                            1.254729  104.142517      83
02_HM                            0.507512   24.868070      49
03_HL                            0.421885    5.062625      12
04_MH                            0.331212   16.891799      51
05_MM                            0.233282   15.396636      66
06_ML                            0.138084   16.846287     122
07_LH                            0.042040   13.116409     312
08_LM                            0.004002    0.012006       3


pid                        4612
daily_new_fans             4612
daily_pid_device_count     4612
daily_pid_radio_score      4596
pid_absolute_score         4612
pid_relative_score          698
daily_short_radio_score     231
presenter_score            4612
bin_device_count           4612
bin_pid_absolute_score     4612
bin_pid_relative_score     4612
dtype: int64

In [64]:
    spc_var = 'daily_pid_radio_score'
    print("社区绝对活跃度: {s} is started".format(s=spc_var))
    oo = df.ix[(df['daily_pid_device_count']>0) &( df['daily_pid_radio_score'])][list_to_qtl]
    
    oo.quantile([0, .1, .2, .3, .4, .5, .6, .7, .8, .9, 0.95, 0.99, 1  ])

    config_dict= {            
            "b1":0.001,
            "b2":0.005,
            "b3":0.1,
            "b4":0.2,
            "b5":0.3,
            "b6":0.4,
            "b7":0.5,
            "b8":0.6
        }
    
    df["""bin_{spc_var}""".format(spc_var=spc_var)] = df.apply (lambda row: func_bin(row, spc_var, config_dict),axis=1)
    
    
    aggregations={spc_var:
                                             {
                                             'avg':'mean',
                                             'sum':'sum',
                                             'cnt_pid':'count'
                                             }
                    }
    df.groupby(["""bin_{spc_var}""".format(spc_var=spc_var)]).agg(aggregations)
    




社区绝对活跃度: daily_pid_radio_score is started


daily_pid_radio_score                    
                                            avg         sum cnt_pid
bin_daily_pid_radio_score                                          
01_HH                                  0.907433  894.728787     986
02_HM                                  0.511986  280.568107     548
03_HL                                  0.433436  104.024591     240
04_MH                                  0.341710  103.196339     302
05_MM                                  0.244015   61.491752     252
06_ML                                  0.147309   11.342760      77
07_LH                                  0.068383    1.709581      25
08_LM                                  0.003195    0.003195       1
09_LL                                  0.000000    0.000000    2165

In [67]:
    spc_var = 'presenter_score'
    print("社区绝对活跃度: {s} is started".format(s=spc_var))
    oo = df.ix[(df['presenter_score']>=0)][list_to_qtl]
    
    oo.quantile([0, .1, .2, .3, .4, .5, .6, .7, .8, .9, 0.95, 0.99, 1  ])

    config_dict= {            
            "b1":0,
            "b2":3,
            "b3":5,
            "b4":10,
            "b5":20,
            "b6":30,
            "b7":40,
            "b8":50
        }
    
    df["""bin_{spc_var}""".format(spc_var=spc_var)] = df.apply (lambda row: func_bin(row, spc_var, config_dict),axis=1)
    
    
    aggregations={spc_var:
                                             {
                                             'avg':'mean',
                                             'sum':'sum',
                                             'cnt_pid':'count'
                                             }
                    }
    df.groupby(["""bin_{spc_var}""".format(spc_var=spc_var)]).agg(aggregations)
    



社区绝对活跃度: presenter_score is started


presenter_score             
                                avg  sum cnt_pid
bin_presenter_score                             
03_HL                     30.000000   30       1
05_MM                     13.538462  176      13
06_ML                      6.111111   55       9
07_LH                      3.500000   56      16
08_LM                      0.018806   86    4573